# PFDA project

***

In [4]:
# import plotting library
import matplotlib.pyplot as plt

# import data analysis library
import pandas as pd

# import statistical library
import seaborn as sns

# import predictive data analysis library
import sklearn